In [10]:
from silero import silero_tts
import numpy as np
import wave
# from pydub import AudioSegment
import json
# from vosk import Model, KaldiRecognizer
import torch
# from IPython.display import Audio, display
import random

from itertools import chain, combinations

Определяем статичные переменные

In [11]:
# пути по умолчанию
folder_train = "../audios_train"
folder_valid = "../audios_valid"
folder_test = "../audios_test"

# устройство для генерации
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# частота дискретизации для генерации аудио
sample_rate = 48000

# количество генераций каждого слова с ошибкой и без
N = 200

# проверяемые звуки Л, Р, Ш, Ж
# + означают ударения (SSML)
# на каждый звук представлен словарь, где ключ - правильное произношение, значение - список возможных ошибок
error_word = {
    "Ж": {
        "ж+аба": ["з+яба", "з+аба", "ш+аба", "х+аба"], # Ж-н
        "наж+и": ["наз+ы", "нас+ы", "наз+и"], # Ж-с
    },
    "Л": {
        "л+ук": ["в+ук", "у+ук", "л+юк", "йук"], # Л-н
        "молок+о": ["мавак+о", "маок+о", "малйак+о"], # Л-с
        "ст+ул": ["ст+уй", "ст+ув", "ст+уу", "ст+уль"], # Л-к
    }, 
    "Р": {
        "рук+а": ["йук+а", "лук+а", "ук+а", "вук+а"], # Р-н
        "бараб+ан": ["балаб+ан", "баваб+ан", "барйаб+ан"], # Р-с
        "топ+ор": ["тап+ол", "тап+охр", "тап+ов", "тап+оы"], # Р-к
    }, 
    "Ш": {
        "ш+ина": ["с+ына", "ф+ына", "х+ына", "щ+ина"], # Ш-н
        "меш+ок": ["мис+ок", "миф+ок", "мих+ок", "мищ+ок"], # Ш-с
        "д+уш": ["д+ус", "д+уф", "д+ух", "д+ущ"], # Ш-к
    }, 
}

In [12]:
def wordSSML(word):
  # случайное время паузы
  time_break1 = random.randint(500, 1000)
  time_break2 = random.randint(500, 1000)

  # возврат текста с необходимами тегами (в соответсвии с SSML)
  return f'<speak>\n<p>\n\t<break time="{time_break1}ms"/>{word}<break time="{time_break2}ms"/>\n</p>\n</speak>'
    

In [13]:
# получаем тензор аудиофайла
def generateAudio(model, text):
  # speaker='random' - автоматическая генерация голоса 
  # put_yo=True - использование SSML (расстановка ударений в тексте)
  # ssml_text - использует текст с SSML разметкой (для обычного текста с ударениями text)
  _audio = model.apply_tts(ssml_text=text,
                          speaker='random',
                          sample_rate=sample_rate,
                          put_accent=True,
                          put_yo=True)
  
  # Преобразовываем тензор в numpy array и нормализуем аплитуду 
  audio_np = _audio.numpy().astype(np.float32)
  max_abs_amplitude = np.max(np.abs(audio_np))
  audio_np /= max_abs_amplitude

  return audio_np

In [14]:
# Сохранение файла в папку
def saveAudioWav(audio, file_name, path):
  with wave.open(f"{path}/{file_name}", 'w') as f:
    # Настройка параметров файла
    f.setnchannels(1)  # 1 канал (моно)
    f.setsampwidth(2)  # 2 байта на сэмпл
    f.setframerate(sample_rate)  # Частота дискретизации
    f.setcomptype('NONE', 'not compressed')  # Без компрессии

    # Преобразование и запись в файл
    audio_int16 = np.int16(audio * 32767)  # Приведение к 16-битному формату
    f.writeframes(audio_int16.tobytes())

In [15]:
def generatorMain(model, path, N):
  index = 0
  # обходим каждый звук
  for sound, s_values in error_word.items():
    # обходим значения звука
    for word, word_errors in s_values.items():
      for _ in range(N):
        # генерируем нормальное произношение
        text = wordSSML(word)
        file_name = f'gen_{index}-{sound}_norm.wav'
        _audio = generateAudio(model = model, text = text)
        saveAudioWav(audio=_audio, file_name=file_name, path=path)
        index += 1
        # генерируем произношение с ошибкой
        select_word = random.choice(word_errors)
        text = wordSSML(select_word)
        file_name = f'gen_{index}-{sound}_err.wav'
        _audio = generateAudio(model = model, text = text)
        saveAudioWav(audio=_audio, file_name=file_name, path=path)
        index += 1

Получаем silero модель синтезатора речи

In [16]:
model, example_text = silero_tts(language="ru", speaker="v3_1_ru")
# model.to(device) # silero рекомендует использовать cpu, он установлен по умолчанию

Генерация обучающих данных

In [17]:
generatorMain(model, folder_train, N)

Generated new voice
Generated new voice
Generated new voice
Generated new voice
Generated new voice
Generated new voice
Generated new voice
Generated new voice
Generated new voice
Generated new voice
Generated new voice
Generated new voice
Generated new voice
Generated new voice
Generated new voice
Generated new voice
Generated new voice
Generated new voice
Generated new voice
Generated new voice
Generated new voice
Generated new voice
Generated new voice
Generated new voice
Generated new voice
Generated new voice
Generated new voice
Generated new voice
Generated new voice
Generated new voice
Generated new voice
Generated new voice
Generated new voice
Generated new voice
Generated new voice
Generated new voice
Generated new voice
Generated new voice
Generated new voice
Generated new voice
Generated new voice
Generated new voice
Generated new voice
Generated new voice
Generated new voice
Generated new voice
Generated new voice
Generated new voice
Generated new voice
Generated new voice


Генерация тестовых данных

In [18]:
generatorMain(model, folder_test, N // 2)

Generated new voice
Generated new voice
Generated new voice
Generated new voice
Generated new voice
Generated new voice
Generated new voice
Generated new voice
Generated new voice
Generated new voice
Generated new voice
Generated new voice
Generated new voice
Generated new voice
Generated new voice
Generated new voice
Generated new voice
Generated new voice
Generated new voice
Generated new voice
Generated new voice
Generated new voice
Generated new voice
Generated new voice
Generated new voice
Generated new voice
Generated new voice
Generated new voice
Generated new voice
Generated new voice
Generated new voice
Generated new voice
Generated new voice
Generated new voice
Generated new voice
Generated new voice
Generated new voice
Generated new voice
Generated new voice
Generated new voice
Generated new voice
Generated new voice
Generated new voice
Generated new voice
Generated new voice
Generated new voice
Generated new voice
Generated new voice
Generated new voice
Generated new voice


In [25]:
generatorMain(model, folder_valid, N // 2)

Generated new voice
Generated new voice
Generated new voice
Generated new voice
Generated new voice
Generated new voice
Generated new voice
Generated new voice
Generated new voice
Generated new voice
Generated new voice
Generated new voice
Generated new voice
Generated new voice
Generated new voice
Generated new voice
Generated new voice
Generated new voice
Generated new voice
Generated new voice
Generated new voice
Generated new voice
Generated new voice
Generated new voice
Generated new voice
Generated new voice
Generated new voice
Generated new voice
Generated new voice
Generated new voice
Generated new voice
Generated new voice
Generated new voice
Generated new voice
Generated new voice
Generated new voice
Generated new voice
Generated new voice
Generated new voice
Generated new voice
Generated new voice
Generated new voice
Generated new voice
Generated new voice
Generated new voice
Generated new voice
Generated new voice
Generated new voice
Generated new voice
Generated new voice


# Алгоритм разбиения файла на слова (не используется)

Обработка распозования речи

In [20]:

# audio = AudioSegment.from_file("./test_maks.mp3", format="mp3")
# audio.export("test_maks.wav", format="wav")

# with wave.open("./test_maks.wav", "rb") as audio_file:
#     audio_data = audio_file.readframes(audio_file.getnframes())

# # получем данные аудиофайла
# # with wave.open(filename, "rb") as audio_file:
# #     audio_data = audio_file.readframes(audio_file.getnframes())

# # определяем модель распознования речи vosk
# model = Model("vosk-model-small-ru-0.22")

# # получаем распознователь
# recognizer = KaldiRecognizer(model, audio_file.getframerate())
# # устанавливаем параметр получения сведений о словах
# recognizer.SetWords(True)

Получаем тайминги слов

In [21]:
# recognizer.AcceptWaveform(audio_data)
# result = json.loads(recognizer.FinalResult())
# result

In [22]:
# from pydub import AudioSegment

# # audio_file = AudioSegment.from_wav(filename)
# audio_file = AudioSegment.from_wav("test_maks.wav")

# for item in result['result']:
#     start_time = item['start'] * 1000
#     end_time = item['end'] * 1000 - 80
#     key = item['word']
#     segment_audio = audio_file[start_time:end_time]
#     segment_audio.export(f'{folder_data}/{key}.wav', format="wav")

# Старые варианты генерации

In [23]:
# def textGenetatorRealLife():
#   # устанавливаем текст с тегом разговора (в соответсвии с SSML)
#   text = "<speak>\n<p>\n"
  
#   with_error = random.random() # шанс ошибки
#   # count_errors - количество ошибочных звуков (25% нарушены 3-4 звука, 35% 1-2 звука, 40% без ошибок)
#   # chance_isol - шанс полной изоляции звука. Для тяжелых детей с 3-4 нарушеными звуками, шанс соответствует 90%, для детей с 1-2 нарушениями 70%
#   if with_error < 0.25: 
#     count_errors = random.randint(3, 4)
#     chance_isol = 0.9
#   elif with_error < 0.6: 
#     count_errors = random.randint(1, 2)
#     chance_isol = 0.7
#   else: 
#     count_errors = 0
#     chance_isol = 0

#   # случайный выбор звуков, которые будут нарушены
#   select_sound_error = random.sample(list(error_word.keys()), count_errors)
#   select_sound_error.sort()
#   # обходим словарь звуков
#   for sound, values_saund in error_word.items():
#     # определяем, будет ли звук изолирован
#     with_isol = random.random() <= chance_isol
#     # список типов звука без ошибки
#     types_without_errors = []
#     if not with_isol:
#       # получаем максимально возможное количество выбраных типов без нарушения
#       max_get_normal = len(values_saund.keys()) - 1
#       # рандомим количество типов без нарушений
#       count_type_normal = random.randint(0, max_get_normal)
#       # получаем типы без нарушения
#       types_without_errors = random.sample(list(values_saund.keys()), count_type_normal)
#     # обходим все типы представления звука (в начале, середине и конце)
#     for soud_type, sound_errors in values_saund.items():
#       # время паузы
#       time_break = random.randint(2000, 3000)
#       # когда тип звука в стоп-листе или сам звук не включен в выбранные звуки, которые должны быть нарушены
#       if soud_type in types_without_errors or sound not in select_sound_error:
#         text += f'\t{soud_type}<break time="{time_break}ms"/>\n'
#       else:
#         # получаем случайное нарушение
#         random_error = random.choice(sound_errors)
#         text += f'\t{random_error}<break time="{time_break}ms"/>\n'
  
#   # закрываем тег разговора
#   text += "</p>\n</speak>"

#   # возврат полученного SSML и звуков с нарушениями
#   return text, select_sound_error

In [24]:
# def textGenetator(sounds):
#   # устанавливаем текст с тегом разговора (в соответсвии с SSML)
#   text = "<speak>\n<p>\n"

#   for sound, values_saund in error_word.items():
    
#     types_with_errors = []
#     if sound in sounds:
#       # получаем максимально возможное количество выбраных положений нарушений
#       max_get_errors = len(values_saund.keys()) - 1
#       # рандомим количество положения нарушения
#       count_type_errors = random.randint(1, max_get_errors)
#       # получаем типы без нарушения
#       types_with_errors = random.sample(list(values_saund.keys()), count_type_errors)

#     for normal_sound, error_sounds in values_saund.items():
#       # время паузы
#       time_break = random.randint(2000, 3000)
#       if normal_sound in types_with_errors:
#         # получаем случайное нарушение
#         random_error = random.choice(error_sounds)
#         text += f'\t{random_error}<break time="{time_break}ms"/>\n'
#       else:
#         text += f'\t{normal_sound}<break time="{time_break}ms"/>\n'

#   # закрываем тег разговора
#   text += "</p>\n</speak>"

#   # возврат полученного SSML
#   return text